<a href="https://colab.research.google.com/github/pankajtandon/my-ai/blob/main/my_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Store keys in google drive to prevent it getting into source control.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install the necessary packages.
!pip install openai -q


In [3]:
import openai
import json
import os

In [8]:
# Make the key available
with open('/content/drive/MyDrive/colab_content/api-keys.txt', 'r') as f:
    api_key_list = f.readlines()
for kv in api_key_list:
    k,v = kv.split('=')
    #print(k, v)
    os.environ[k] = v.strip()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [42]:
GPT_MODEL = "gpt-3.5-turbo-0613"
#GPT_MODEL = "gpt-4"

# This is metadata about the functions that the model will use to determine what to call.
function_meta = [
    {
        "name": "get_my_bank_balance",
        "description": "Returns the amount of money I have in my bank account",
        "parameters": {
            "type": "object",
            "properties": {
                "accountId": {
                    "type": "string",
                    "description": "the account id that holds all my money"
                },
                "routingNumber": {
                    "type": "string",
                    "description": "the routing number of the bank that has the account that holds all my money"
                },
            },
            "required": ["accountId", "routingNumber"]
        }
    },
    {
        "name": "get_routing_number_for_bank",
        "description": "Returns the routing number of the bank given the name of the bank",
        "parameters": {
            "type": "object",
            "properties": {
                "bankName": {
                    "type": "string",
                    "description": "the name of the bank"
                }
            },
            "required": ["bankName"]
        }
    },
    {
        "name": "get_name_of_my_bank",
        "description": "Returns bank name where my money is saved!",
        "parameters": {
            "type": "object",
            "properties": {
            }
        }
    },
    {
        "name": "get_my_account_number",
        "description": "Returns my account number",
        "parameters": {
            "type": "object",
            "properties": {
            }
        }
    }
]


# Implementations of functions

def get_my_bank_balance(accountId, routingNumber):
    # go out and get it!
    # For now:
    # print("---->Returning bank balance for accountId", accountId, " and routingNumber", routingNumber)
    return {"balance" : "12500", "currency" : "USD"}

def get_routing_number_for_bank(bankName) :
    # go out and get it!
    # For now:
    # print("--->Returning routingNumber for bank", bankName)
    return {"routingNumber": "ROUTING_1234"}

def get_my_account_number() :
    # go out and get it!
    # For now:
    # print("--->Returning account number")
    return {"accountId": "AC_NUM_1234"}

def get_name_of_my_bank():
    # go out and get it!
    # For now:
    # print("---->Returning a random bank name")
    return {"bankName": "Chase"}



# Calls to implementations based on models response
def call_the_implemented_function(chat_response):
    function_call = chat_response["choices"][0]["message"]["function_call"]
    function_name = function_call["name"]
    #print("--------->", function_name)
    arguments = function_call["arguments"]
    if function_name == "get_name_of_my_bank":
        return get_name_of_my_bank()
    elif function_name == "get_routing_number_for_bank":
        bankName = eval(arguments).get("bankName")
        return get_routing_number_for_bank(bankName)
    elif function_name == "get_my_account_number":
        return get_my_account_number()
    elif function_name == "get_my_bank_balance":
        routingNumber = eval(arguments).get("routingNumber")
        accountId = eval(arguments).get("accountId")
        return get_my_bank_balance(accountId, routingNumber)
    else:
        return


def initiate_chat(query):
    messages = [{"role": "user", "content": query}]
    loop_check = True
    while loop_check:
        response = openai.ChatCompletion.create(
            model= GPT_MODEL,
            messages=messages,
            functions = function_meta,
            function_call="auto"
        )
        loop_check = (response["choices"][0]["finish_reason"] == "function_call")
        #print ("Model response", response)
        if loop_check:
          print("Called function", response["choices"][0]["message"]["function_call"]["name"], "with arguments", response["choices"][0]["message"]["function_call"]["arguments"])
          function_response = call_the_implemented_function(response)
          print("Function returned", json.dumps(function_response))
          messages.append({
              "role": "function",
              "name": response["choices"][0]["message"]["function_call"]["name"],
              "content": json.dumps(function_response)
          })
          # print("messages: ", messages)
    else:
        print("Final answer", response["choices"][0]["message"]["content"])

# This uses a random accountId and a routing number instead of using my custom functions
# query = "What are my savings in the bank that I store my money in?"

# Works! -> "Your account number is AC_NUM_1234 and the routing number for your bank is 1234ABC. The savings in your bank account is $12,500."
# query = "First determine my account number and my routing number and then use that info to determine what are my savings in the bank that I store my money in?"

# Partial: Got my bank name but used fake account number and routing number: "Your savings in the bank where you store your money, which is Chase, is $12,500."
# query = "What are my savings in the bank that I store my money in? Don't make assumptions about my accountId and my routing number"

# Works! -> "Based on the information provided, your account number is AC_NUM_1234 and your routing number is 1234ABC. Your current savings in the bank is $12,500.\n\nTo determine if you can buy a car worth $13,000 with your current savings, we need to compare the savings with the cost of the car. \n\nSince your savings is $12,500, which is less than $13,000, you will not be able to buy the car with your current savings. You need an additional $500 to meet the cost of the car."
# query = "If all I wanted in life was a car worth $13000, could I buy that with my current savings? First determine my account number, bank name and my routing number and then use that info to determine what are my savings in the bank. Why or why not?"

# query = "If all I wanted in life was a car worth $13000, could I buy that with my current savings?"

# query = "Based on the my actual account number and actual routing number of my bank, can you determine that if all I wanted in life was a car worth $13000, could I buy that with my current savings?"

# This goes directly for get_my_bank_balance with fictitous ac num and routing num.
#initiate_chat("Based on my bank's name, accountId and routing number, determine if I can purchase a car worth $12570")


initiate_chat("First determine my account number and my routing number and then use that info to determine what are my savings in the bank that I store my money in?")


Called function get_my_account_number with arguments {}
Function returned {"accountId": "AC_NUM_1234"}
Called function get_routing_number_for_bank with arguments {"bankName": "My Bank"}
Function returned {"routingNumber": "ROUTING_1234"}
Called function get_my_bank_balance with arguments {
"accountId": "AC_NUM_1234",
"routingNumber": "ROUTING_1234"
}
Function returned {"balance": "12500", "currency": "USD"}
Final answer Your account number is AC_NUM_1234 and the routing number for your bank is ROUTING_1234. The current savings in your bank account is $12,500.
